### Import bibliotek

In [1]:
import pandas as pd
import numpy as np

### Import danych

In [2]:
data = pd.DataFrame({
    'gender': ['male', 'female', 'male', 'female', 'female'],
    'age': [20, 35, 50, 40, 25],
    'height': [170, 165, 180, 160, 175],
    'weight': [90, 65, 82, 70, 72],
    'seating_work': [1, 0, 1, 1, 0],
    'hypertension': [1, 0, 1, 0, 1],
})

data

,gender,age,height,weight,seating_work,hypertension
0,male,20,170,90,1,1
1,female,35,165,65,0,0
2,male,50,180,82,1,1
3,female,40,160,70,1,0
4,female,25,175,72,0,1


In [3]:
np.where(data["age"]>20, 1, 0)

array([0, 1, 1, 1, 1])

### Klasa Node

In [4]:
# Klasa Node reprezentuje węzeł w drzewie decyzyjnym
class Node:

    def __init__(
            self,
            feature=None,         # nazwa cechy, po której następuje podział (np. 'wiek', 'wzrost')
            split_value=None,     # wartość podziału (np. wiek > 30)
            is_binary=False,      # czy cecha jest dychotomiczna (True/False) czy ciągła (np. wiek)
            left=None,            # lewe poddrzewo (niespełniony warunek podziału) -> zmieniliśmy, bo niżej w kodzie był problem, bo w drugą stronę pisaliśmy xd
            right=None,           # prawe poddrzewo (spełniony warunek podziału)
            value=None,           # wartość przewidywana w liściu
            parent = None):       # rodzic węzła (None dla korzenia)
            # left, right i value jednocześnie nie mogą być różne od None
        self.feature = feature
        self.split_value = split_value
        self.is_binary = is_binary
        self.left = left
        self.right = right
        self.value = value
        self.parent = parent

    @property # wytłumaczyłam sobie niżej w tekście, czym jest property
    def is_leaf(self):
        # Sprawdza czy węzeł jest liściem (nie ma dzieci)
        return self.left is None and self.right is None
    
    @property
    def is_root(self):
        # Sprawdza czy węzeł jest korzeniem (nie ma rodzica)
        return self.parent is None
    
    def predict(self, x: pd.Series): # ok, x: to taki hint dla pythona, ale i tak nic sobie z nim nie zrobi, czyli to hint dla mnie, dziękuję
        # Metoda dokonująca predykcji dla pojedynczego przykładu x
        # x to wiersz danych jako pandas Series
        
        # Jeśli jesteśmy w liściu, zwracamy jego wartość
        if self.is_leaf:
            return self.value
            
        # usunęliśmy if self.is_binary, można tak?
        if x[self.feature] > self.split_value:
            return self.right.predict(x)
        else:
            return self.left.predict(x)
            

### 

In [5]:
def gini_impurity_for_feature(df: pd.DataFrame, feature: str, target: str="hypertension") -> tuple[float, any]:
    if len(df[feature].unique()) == 1:
        return 9999.0, df[feature].iloc[0]
    if len(df[feature].unique()) == 2:
        return get_gini_total(df[feature], df[target]), df[feature].min()
    # trzeci i na razie ostatni przypadek
    unique_sorted = sorted(df[feature].unique())
    gini_with_split_value = []
    for i in range(len(unique_sorted)-1):
        split_value = unique_sorted[i]
        binary = np.where(df[feature]>split_value, 0, 1)
        gini_per_split_value = get_gini_total(binary, df[target])
        gini_with_split_value.append((gini_per_split_value, split_value))
    return min(gini_with_split_value) # min działa też na tuplach i tutaj zwraca tuplę z najmniejszą liczbą na pierwszym miejscu

def get_gini_total(feature, target):
    # tworzymy tablicę kontyngencji
        contingency = pd.crosstab(feature, target)
        # zliczamy liczebność - liczba wierszy w wejściowej df
        total = contingency.sum().sum()
        # obliczamy gini dla każdego wiersza czyli dla każdej kategorii
        gini_per_category = 1 - (contingency.div(contingency.sum(axis=1), axis=0) ** 2).sum(axis=1)
        # obliczamy gini dla całej kolumny jako  średnia ważona
        gini_total = (gini_per_category * (contingency.sum(axis=1) / total)).sum()
        if gini_total == 0:
            return 9999
        return gini_total

In [6]:
def lowest_gini(df: pd.DataFrame, target: str, used_cols: list):
    # lista kolumn z df bez kolumny bez targetu
    features = [feature for feature in df.columns if feature != target and feature not in used_cols]
    # szukamy feature, dla którego gini impurity jest najmniejszy przy podziale wg targutu
    # na pamiątkę <3 -> best_feature = min(features, key=lambda feature: gini_impurity_for_feature(df, feature, target))  # XDDDD wooooow
    lista = [(*gini_impurity_for_feature(df, f, target), f) for f in features] # lista tupli postaci: (gini, split_value, feature)
    best = min(lista, key=lambda x: x[0]) # best to tupla
    return best[2], best[1] # feature, split_value

In [7]:
def build_tree_(df, target, used_cols):
    # na razie zmienne tylko dla których len(df[feature].unique())==2
    # obsługujemy przypadek, że df od razu jest liściem:
# sprawdzić, czy df jest puste -> defensive coding
    if len(df[target].unique()) == 1:
        print("first if") # czyli mamy df z samymi 0 lub 1 w kolumnie "hypertension"
        return Node(value=df[target].iloc[0]) # bierzemy pierwszą wartość z Series, bo wszystkie takie same
    # sprawdzamy, czy zostały jeszcze jakieś kolumny niewykorzystane
    if len(used_cols) == len(df.columns) - 1:
        mode = df[target].mode()[0]
        return Node(value=mode)
    best_feature, split_value = lowest_gini(df, target, used_cols)
    print("Best feature:", best_feature,",", "split value:", split_value)
    is_binary = len(df[best_feature].unique()) <= 2
    if is_binary:
        used_cols.append(best_feature)
    # dzielimy na dwie df w zależności od spełnienia warunku
    right_df = df[df[best_feature]>split_value]
    left_df = df[df[best_feature]<=split_value]
    # potwierdzimy sobie, co wyprintuje
    print(right_df)
    print(left_df)
    # Jeśli któraś gałąź pusta → liść z najczęstszą klasą
    if len(left_df) == 0 or len(right_df) == 0:
        majority = df[target].mode()[0]
        print("majority")
        return Node(value=majority)
    right_child = build_tree_(right_df, target, used_cols)
    left_child = build_tree_(left_df, target, used_cols)
    current_node = Node(feature=best_feature, split_value=split_value, is_binary=is_binary, left=left_child, right=right_child)
    left_child.parent = current_node
    right_child.parent = current_node
    return current_node

In [8]:
def build_tree(df: pd.DataFrame, target: str="hypertension"):
    used_cols = []
    return build_tree_(df, target, used_cols)

In [9]:
# fajny feature
lista = [(1, 155), (388, 10), (66, 15)]
min(lista, key=lambda x : x[1])

(388, 10)

In [13]:
tree = build_tree(data)

Best feature: gender , split value: female
  gender  age  height  weight  seating_work  hypertension
0   male   20     170      90             1             1
2   male   50     180      82             1             1
   gender  age  height  weight  seating_work  hypertension
1  female   35     165      65             0             0
3  female   40     160      70             1             0
4  female   25     175      72             0             1
first if
Best feature: age , split value: 35
   gender  age  height  weight  seating_work  hypertension
3  female   40     160      70             1             0
   gender  age  height  weight  seating_work  hypertension
1  female   35     165      65             0             0
4  female   25     175      72             0             1
first if
Best feature: age , split value: 25
   gender  age  height  weight  seating_work  hypertension
1  female   35     165      65             0             0
   gender  age  height  weight  seating_work

In [14]:
for i in range(len(data)):
    print(tree.predict(data.iloc[i]))

1
0
1
0
1
